In [6]:
import json
with open('total.json') as f:
    data = json.load(f)

In [17]:
import os
import requests
import json
from dotenv import load_dotenv

load_dotenv()


REST_API_KEY = os.getenv('KAKAO_API')
URL = "https://dapi.kakao.com/v2/local/search/keyword.json"

In [25]:
def search(query: str) -> dict:
    headers = {
    "Authorization": f"KakaoAK {REST_API_KEY}"
    }

    params = {
        "query": query,
        "x": "127.09761630337", # 송파구
        "y": "37.51232891786",
        "radius": 20000,
        "page":1
    }

    response = requests.get(URL, headers=headers, params=params)
    os.makedirs("./kakao_data", exist_ok=True)
    if response.status_code == 200:
        data = response.json()
        # print(json.dumps(response.json(), indent=4, ensure_ascii=False))
        return data
    else:
        print(f"Error {response.status_code}: {response.text}")
        return False

In [35]:
from random import randint

data_with_meta = {'total_data': []}

for gender in data:
    data_gender = data[gender]
    for age in data_gender:
        data_gender_age = data_gender[age]
        for single_data in data_gender_age:
            if '테마파크' in single_data['category_name']:
                price = 50_000
            else:
                price = randint(1_000, 20_000) // 100 * 100
            # print(single_data)
            search_data = search(single_data['place_name'])['documents']
            # print(search_data)
            if not search_data:
                continue
            first_one = search_data[0]
            new_one = dict()
            new_one['category_name'] = first_one['category_name']
            new_one['place_name'] = first_one['place_name']
            new_one['road_address_name'] = first_one['road_address_name']
            new_one['price'] = price
            data_with_meta['total_data'].append(new_one)
            # break

In [44]:
# from pprint import pprint
# pprint(data_with_meta)
with open('data_with_meta.json', 'w', encoding='UTF-8') as outfile:
    json.dump(data_with_meta, outfile, indent=4, ensure_ascii=False)